In [87]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score

In [88]:
ames = pd.read_csv("AmesHousing.csv")
ames = ames.dropna(subset=['SalePrice', "Bldg Type", "Gr Liv Area", "TotRms AbvGrd"])
ames.head()



,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [89]:
lr = LinearRegression()
X = ames.drop('SalePrice', axis = 1)
y = ames["SalePrice"]

In [90]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline1 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline1

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

Consider four possible models for predicting house prices:

Using only the size and number of rooms.

Using size, number of rooms, and building type.

Using size and building type, and their interaction.

Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

In [91]:
fitted1 = lr_pipeline1.fit(X_train, y_train)
pred1 = fitted1.predict(X_test)
rmse1 = sqrt(mean_squared_error(pred1,y_test))
rmse1

62164.94789903393

In [92]:
scores1 = cross_val_score(lr_pipeline1, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores1).mean()

3136138908.170903

In [93]:
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline2 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [94]:
fitted2 = lr_pipeline2.fit(X_train, y_train)
pred2 = fitted2.predict(X_test)
rmse2 = sqrt(mean_squared_error(pred2,y_test))
rmse2

59939.254533124506

In [95]:
scores2 = cross_val_score(lr_pipeline2, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores2).mean()

2951993958.10073

In [104]:
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])],
  remainder = "passthrough"
).set_output(transform = "pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_1Fam"]),
  ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline3 = Pipeline(
  [("preprocessing1", ct_dummies),
   ("preprocessing2", ct_inter),
  ("linear_regression", LinearRegression())]
)
ct_inter.fit_transform(X_train)

ValueError: A given column is not a column of the dataframe

In [97]:
fitted3 = lr_pipeline3.fit(X_train, y_train)
pred3 = fitted3.predict(X_test)
rmse3 = sqrt(mean_squared_error(pred3,y_test))
rmse3

60810.548042026116

In [98]:
scores3 = cross_val_score(lr_pipeline3, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores3).mean()

3132206753.875458

In [99]:
ct = ColumnTransformer(
    [("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
     ("degreesize", PolynomialFeatures(degree=5, interaction_only=False), ["Gr Liv Area"]),
     ("degreesize2", PolynomialFeatures(degree=5, interaction_only=False), ["TotRms AbvGrd"])],
    remainder="drop"
).set_output(transform = "pandas")

lr_pipeline4 = Pipeline(
  [("preprocessing1", ct),
  ("linear_regression", LinearRegression())]
)
lr_pipeline4

Pipeline(steps=[('preprocessing1',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('degreesize',
                                                  PolynomialFeatures(degree=5),
                                                  ['Gr Liv Area']),
                                                 ('degreesize2',
                                                  PolynomialFeatures(degree=5),
                                                  ['TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [100]:
ct.fit_transform(X_train)

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,degreesize__1,degreesize__Gr Liv Area,degreesize__Gr Liv Area^2,degreesize__Gr Liv Area^3,degreesize__Gr Liv Area^4,degreesize__Gr Liv Area^5,degreesize2__1,degreesize2__TotRms AbvGrd,degreesize2__TotRms AbvGrd^2,degreesize2__TotRms AbvGrd^3,degreesize2__TotRms AbvGrd^4,degreesize2__TotRms AbvGrd^5
1347,1.0,0.0,0.0,0.0,0.0,1.0,1774.0,3147076.0,5.582913e+09,9.904087e+12,1.756985e+16,1.0,8.0,64.0,512.0,4096.0,32768.0
1720,0.0,0.0,0.0,1.0,0.0,1.0,1554.0,2414916.0,3.752779e+09,5.831819e+12,9.062647e+15,1.0,6.0,36.0,216.0,1296.0,7776.0
600,1.0,0.0,0.0,0.0,0.0,1.0,864.0,746496.0,6.449725e+08,5.572563e+11,4.814694e+14,1.0,5.0,25.0,125.0,625.0,3125.0
1079,0.0,0.0,0.0,0.0,1.0,1.0,1358.0,1844164.0,2.504375e+09,3.400941e+12,4.618478e+15,1.0,6.0,36.0,216.0,1296.0,7776.0
1618,1.0,0.0,0.0,0.0,0.0,1.0,1600.0,2560000.0,4.096000e+09,6.553600e+12,1.048576e+16,1.0,7.0,49.0,343.0,2401.0,16807.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,0.0,0.0,0.0,0.0,1.0,1.0,1767.0,3122289.0,5.517085e+09,9.748689e+12,1.722593e+16,1.0,5.0,25.0,125.0,625.0,3125.0
1710,1.0,0.0,0.0,0.0,0.0,1.0,2374.0,5635876.0,1.337957e+10,3.176310e+13,7.540560e+16,1.0,10.0,100.0,1000.0,10000.0,100000.0
1949,1.0,0.0,0.0,0.0,0.0,1.0,968.0,937024.0,9.070392e+08,8.780140e+11,8.499175e+14,1.0,5.0,25.0,125.0,625.0,3125.0
2434,1.0,0.0,0.0,0.0,0.0,1.0,1936.0,3748096.0,7.256314e+09,1.404822e+13,2.719736e+16,1.0,7.0,49.0,343.0,2401.0,16807.0


In [101]:
fitted4 = lr_pipeline4.fit(X_train, y_train)
pred4 = fitted4.predict(X_test)
rmse4 = sqrt(mean_squared_error(pred4,y_test))
rmse4

60344.359232014365

In [102]:
scores4 = cross_val_score(lr_pipeline4, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores4).mean()

3198787103.668873

In [103]:
from sklearn.model_selection import GridSearchCV

ct_poly = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("polynomial", PolynomialFeatures(), ["Gr Liv Area"])
  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__polynomial__degree': np.arange(1, 10)}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='r2')

In [82]:
ct_poly.fit_transform(X_train)

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,polynomial__1,polynomial__Gr Liv Area,polynomial__Gr Liv Area^2
1347,1.0,0.0,0.0,0.0,0.0,1.0,1774.0,3147076.0
1720,0.0,0.0,0.0,1.0,0.0,1.0,1554.0,2414916.0
600,1.0,0.0,0.0,0.0,0.0,1.0,864.0,746496.0
1079,0.0,0.0,0.0,0.0,1.0,1.0,1358.0,1844164.0
1618,1.0,0.0,0.0,0.0,0.0,1.0,1600.0,2560000.0
...,...,...,...,...,...,...,...,...
2327,0.0,0.0,0.0,0.0,1.0,1.0,1767.0,3122289.0
1710,1.0,0.0,0.0,0.0,0.0,1.0,2374.0,5635876.0
1949,1.0,0.0,0.0,0.0,0.0,1.0,968.0,937024.0
2434,1.0,0.0,0.0,0.0,0.0,1.0,1936.0,3748096.0
